#JOTHIKA S
#START TECH ACADEMY
#TASK 2 PROBLEM 4
#QUESTION :

                  #To create a deep learning model that can detect if osteoarthritis is present or not in a given knee X-ray image



In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from glob import glob
import os
import random
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model, Model, Sequential
import cv2,os
from google.colab import drive
drive.mount('/content/drive')
data_path='/content/drive/MyDrive/Osteoarthritis/Osteoarthritis_Assignment_dataset'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
IMG_height = 224
IMG_width = 224
val_path=r'/content/drive/MyDrive/Osteoarthritis/Osteoarthritis_Assignment_dataset/Valid'
test_path=r'/content/drive/MyDrive/Osteoarthritis/Osteoarthritis_Assignment_dataset/test'
train_path=r'/content/drive/MyDrive/Osteoarthritis/Osteoarthritis_Assignment_dataset/train'

In [ ]:
classes = os.listdir(train_path)

# Counting total number of images for training and valdation
num_train = len(glob(train_path + '/*/*'))
num_val = len(glob(val_path + '/*/*'))
print('Classes : ', classes)
print('Number of training images : {num_train}\nNumber of validation images : {num_val}')

Classes :  ['Osteoarthritis', 'Normal']
Number of training images : {num_train}
Number of validation images : {num_val}


In [ ]:
# Loading images from image paths
def parse_image(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.resize(image, [IMG_height, IMG_width])
    image = tf.cast(image, tf.float32) / 255.0
    return image

In [ ]:
def make_dataset(path, batch_size):
    
    # Collecting all filepath in a directory
    filenames = glob(path + '/*/*')
    # Shuffling the dataset
    random.shuffle(filenames)
    # Extraction labels from fil paths(as numbers Normal:0, Osteoarthritis:1)
    labels = [classes.index(name.split(os.path.sep)[-2]) for name in filenames]

    # Creating instance of tf.data.dataset from filenames
    filenames_ds = tf.data.Dataset.from_tensor_slices(filenames)
    # loading files
    images_ds = filenames_ds.map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    labels_ds = tf.data.Dataset.from_tensor_slices(labels)
    images_ds = tf.data.Dataset.zip((images_ds, labels_ds))
    images_ds = images_ds.shuffle(buffer_size=100)
    images_ds = images_ds.batch(batch_size)
    images_ds = images_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return images_ds

In [ ]:
%%time
ds_train = make_dataset(train_path, 32)
ds_val = make_dataset(train_path, 32)
ds_test = make_dataset(train_path, 128)


CPU times: user 226 ms, sys: 28.3 ms, total: 254 ms
Wall time: 588 ms


In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor="val_loss",
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-2,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=5,
        verbose=1,
    )
]

In [ ]:
def build_model_A():
    batch_size = 32
    num_classes = 2

    i = Input(shape=(224, 224, 1))
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
    x = BatchNormalization()(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    x = Dropout(0.2)(x)

    # Hidden layer
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)

    # last hidden layer i.e.. output layer
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(i, x)

    # model description
    model.summary()

    model.compile(optimizer=keras.optimizers.Adam(3e-4),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
model = build_model_A()
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 32)      320       
                                                                 
 batch_normalization (BatchN  (None, 224, 224, 32)     128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 32)      9248      
                                                                 
 batch_normalization_1 (Batc  (None, 224, 224, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0     

In [13]:
# training model
model.fit(ds_train, epochs=20, steps_per_epoch=math.ceil(num_train/32), 
          verbose=1, callbacks=callbacks, validation_data=ds_val,
         validation_steps=math.ceil(num_val/32))

Epoch 1/20
74/74 [==============================] - 989s 13s/step - loss: 0.4961 - accuracy: 0.7626 - val_loss: 77.6298 - val_accuracy: 0.3482
Epoch 2/20
74/74 [==============================] - 980s 13s/step - loss: 0.3926 - accuracy: 0.8200 - val_loss: 76.0180 - val_accuracy: 0.3497
Epoch 3/20
74/74 [==============================] - 982s 13s/step - loss: 0.3093 - accuracy: 0.8591 - val_loss: 80.0467 - val_accuracy: 0.3467
Epoch 4/20
74/74 [==============================] - 990s 13s/step - loss: 0.2583 - accuracy: 0.8889 - val_loss: 52.6046 - val_accuracy: 0.3527
Epoch 5/20
74/74 [==============================] - 987s 13s/step - loss: 0.2170 - accuracy: 0.9072 - val_loss: 37.3826 - val_accuracy: 0.3839
Epoch 6/20
74/74 [==============================] - 987s 13s/step - loss: 0.1646 - accuracy: 0.9328 - val_loss: 16.3798 - val_accuracy: 0.5982
Epoch 7/20
74/74 [==============================] - 993s 13s/step - loss: 0.1492 - accuracy: 0.9370 - val_loss: 7.0970 - val_accuracy: 0.6548


In [14]:
model.save('model.h5')

In [15]:
# evaluating model
model.evaluate(ds_test)

19/19 [==============================] - 227s 11s/step - loss: 0.0763 - accuracy: 0.9774


[0.07634277641773224, 0.9774467945098877]

In [16]:
# saving model
model.save('Osteoarthritis_Classifier.model', save_format='h5')